<a href="https://colab.research.google.com/github/sravani-bogadi-maf/License-plate-detection-main/blob/main/Car_License_Detection_using_Yolo_v10_custom_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jun 13 07:54:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!unzip /content/yolov10.zip

Archive:  /content/yolov10.zip
453c6e38a51e9d1d5a2aa5fb7f1014a711913397
   creating: yolov10-main/
  inflating: yolov10-main/.gitignore  
  inflating: yolov10-main/.pre-commit-config.yaml  
  inflating: yolov10-main/CONTRIBUTING.md  
  inflating: yolov10-main/LICENSE    
  inflating: yolov10-main/README.md  
  inflating: yolov10-main/app.py     
   creating: yolov10-main/docker/
  inflating: yolov10-main/docker/Dockerfile  
  inflating: yolov10-main/docker/Dockerfile-arm64  
  inflating: yolov10-main/docker/Dockerfile-conda  
  inflating: yolov10-main/docker/Dockerfile-cpu  
  inflating: yolov10-main/docker/Dockerfile-jetson  
  inflating: yolov10-main/docker/Dockerfile-python  
  inflating: yolov10-main/docker/Dockerfile-runner  
   creating: yolov10-main/docs/
  inflating: yolov10-main/docs/README.md  
  inflating: yolov10-main/docs/build_docs.py  
  inflating: yolov10-main/docs/build_reference.py  
  inflating: yolov10-main/docs/coming_soon_template.md  
   creating: yolov10-main/do

In [3]:
%cd /content/yolov10/

/content/yolov10


In [4]:
!pwd

/content/yolov10


In [5]:
!pip install .

Processing /content/yolov10
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.9 MB/s eta 0:00:00
  Created wheel for ultralytics: filena

In [6]:
import os
import urllib.request

# Create a directory for the weights in the current working directory
weights_dir = os.path.join(os.getcwd(), "weights")
os.makedirs(weights_dir, exist_ok=True)

# URLs of the weight files
urls = [
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10m.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt",
]

# Download each file
for url in urls:
    file_name = os.path.join(weights_dir, os.path.basename(url))
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded {file_name}")

Downloaded /content/yolov10/weights/yolov10n.pt
Downloaded /content/yolov10/weights/yolov10s.pt
Downloaded /content/yolov10/weights/yolov10m.pt
Downloaded /content/yolov10/weights/yolov10b.pt
Downloaded /content/yolov10/weights/yolov10x.pt


In [7]:
file_path = "/usr/local/lib/python3.11/dist-packages/ultralytics/nn/tasks.py"
with open(file_path, 'r') as file:
    lines = file.readlines()

modified_lines = []
in_function = False
for line in lines:
    if "def torch_safe_load(weight):" in line:
        in_function = True
        modified_lines.append(line)
        continue
    if in_function and "torch.load(" in line and "map_location=" in line and "weights_only=" not in line:
        modified_line = line.replace("map_location=\"cpu\")", "map_location=\"cpu\", weights_only=False)")
        modified_line = modified_line.replace("map_location='cpu')", "map_location='cpu', weights_only=False)")
        modified_lines.append(modified_line)
        in_function = False
    else:
        modified_lines.append(line)
with open(file_path, 'w') as file:
    file.writelines(modified_lines)

print(f"Attempted to modify {file_path}. Please check for errors, especially permission errors.")

Attempted to modify /usr/local/lib/python3.11/dist-packages/ultralytics/nn/tasks.py. Please check for errors, especially permission errors.


In [ ]:
!yolo task=detect mode=train epochs=100 batch=16 plots=True model=weights/ data=data.yaml

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
New https://py

In [7]:
import cv2
import os

frames_dir_v5 = os.path.join(os.getcwd(), "frames_V5")
os.makedirs(frames_dir_v5, exist_ok=True)

video_path_v5 = "/content/yolov10/V5.avi"
cap_v5 = cv2.VideoCapture(video_path_v5)

frame_count_v5 = 0
print(f"Extracting frames from {video_path_v5}...")
while cap_v5.isOpened():
    ret_v5, frame_v5 = cap_v5.read()
    if not ret_v5:
        break
    frame_count_v5 += 1
    frame_filename_v5 = os.path.join(frames_dir_v5, f"frame_V5_{frame_count_v5:05d}.jpg")
    cv2.imwrite(frame_filename_v5, frame_v5)

cap_v5.release()
cv2.destroyAllWindows()
print(f"Finished extracting {frame_count_v5} frames to {frames_dir_v5}")

Extracting frames from /content/yolov10/V5.avi...


error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [10]:
import torch
import sys

# Add the directory containing ultralytics to the Python path
# This might be necessary for torch.serialization to find the module
sys.path.append('/content/yolov10')

# Import the necessary class
from ultralytics.nn.tasks import DetectionModel

# Add the class to the safe globals
torch.serialization.add_safe_globals([DetectionModel])

print("Attempted to add DetectionModel to safe globals.")

Attempted to add DetectionModel to safe globals.


In [16]:
data_yaml_content = """
train: /content/yolov10/frames_V5
val: /content/yolov10/frames_V5

nc: 1
names: ['License-Plate']

# download: https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml_content)

print("Updated data.yaml with class information. Please ensure your annotation files use this class name.")

Updated data.yaml with class information. Please ensure your annotation files use this class name.


In [ ]:
!yolo task=detect mode=train epochs=100 batch=16 plots=True model=weights/yolov10n.pt data=data.yaml name=license_plate_training_v5_frames